# Suvivor US keywords
> How often was show [jargon words](https://docs.google.com/spreadsheets/d/1owUkwauJE24EkMUmVyDl7CbnumOygGfC6BufG7Vspd8/edit?gid=0#gid=0) used by season and episode according to the transcripts

#### Load Python tools and Jupyter config

In [1]:
import os
import re
import json
import pandas as pd
import altair as alt
from collections import defaultdict
import altair_stiles as altstiles
from google.oauth2.service_account import Credentials

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

---

## Process

#### Read the defined keywords

In [ ]:
with open('../data/processed/transcripts/survivor_transcript_words.json', 'r') as f:
    keywords = json.load(f)

In [ ]:
keywords_df = pd.DataFrame(keywords)

#### Read the transcripts

In [4]:
transcripts_df = pd.read_json('../data/raw/other/transcripts/transcripts.json')

#### Function to count occurrences of a word

In [5]:
def count_keywords(text, word, alternates):
    text = text.lower()
    # Use \b word boundaries to ensure whole word matches
    count = len(re.findall(r'\b' + re.escape(word.lower()) + r'\b', text))
    for alt in alternates.split(', '):
        if alt:
            count += len(re.findall(r'\b' + re.escape(alt.lower()) + r'\b', text))
    return count

#### Process each transcript

In [6]:
keyword_counts = defaultdict(lambda: defaultdict(int))
total_word_counts = defaultdict(int)

for index, row in transcripts_df.iterrows():
    transcript = row['transcript']
    season, episode, title = row['season'], row['episode'], row['title']
    
    # Calculate total word count
    total_words = len(re.findall(r'\b\w+\b', transcript.lower()))
    total_word_counts[(season, episode, title)] = total_words
    
    for keyword in keywords:
        word = keyword['word']
        alternates = keyword['alternates']
        count = count_keywords(transcript, word, alternates)
        keyword_counts[(season, episode, title)][word] += count

#### Convert the results to a list of dictionaries for JSON output

In [7]:
results = []
for key, counts in keyword_counts.items():
    season, episode, title = key
    counts_dict = {word: count for word, count in counts.items()}
    total_words = total_word_counts[key]
    normalized_counts = {word: (count / total_words) * 100 for word, count in counts.items()}
    
    results.append({
        'season': season,
        'episode': episode,
        'title': title,
        'counts': counts_dict,
        'total_words': total_words,
        'normalized_counts': normalized_counts
    })

#### Convert results to a DataFrame

In [ ]:
counts_list = []
for result in results:
    season = result['season']
    episode = result['episode']
    title = result['title']
    total_words = result['total_words']
    for word, count in result['counts'].items():
        normalized_count = result['normalized_counts'][word]
        counts_list.append({
            'season': season,
            'episode': episode,
            'title': title,
            'word': word,
            'count': count,
            'total_words': total_words,
            'normalized_count': normalized_count
        })

counts_df = pd.DataFrame(counts_list)

#### Aggregate by season

In [48]:
season_agg_src = counts_df.groupby(['season', 'word']).agg({'count': 'sum', 'total_words': 'sum'}).reset_index()
season_agg_src['normalized_count'] = round((season_agg_df['count'] / season_agg_df['total_words']) * 1000,2)

#### Include word categories

In [53]:
season_agg_df = pd.merge(season_agg_src, keywords_df[['word', 'category']], on='word')

In [55]:
season_agg_df.head()

,season,word,count,total_words,normalized_count,category
0,1,advantage,6,91673,0.07,game play
1,1,alliance,121,91673,1.32,game play
2,1,auction,0,91673,0.00,challenges
3,1,beach,42,91673,0.46,camp life
4,1,beer,19,91673,0.21,food/drink


#### Aggregate for the entire series

In [56]:
series_agg_df = counts_df.groupby('word').agg({'count': 'sum', 'total_words': 'sum'}).reset_index()
series_agg_df['normalized_count'] = round((series_agg_df['count'] / series_agg_df['total_words']) * 1000,2)
series_agg_df = series_agg_df.sort_values('count', ascending=False).reset_index(drop=True)

#### Limit super frequent words (Jeff, tribe, etc)

In [57]:
most_common_keywords = series_agg_df.word.head().to_list()
least_common_keywords = series_agg_df.word.tail(10).to_list()
season_agg_simple_df = season_agg_df.query(f'~word.isin({most_common_keywords}) and ~word.isin({least_common_keywords})')

---

## Charts

#### Heatmaps to show word counts

In [61]:
alt.Chart(season_agg_simple_df, padding={'left': 10}).mark_rect().encode(
    alt.X("season:O")
        .axis(
            labelAngle=0,
            labelOverlap=False,
            values=[10, 20, 30, 40]
        ).title('Season'),
    alt.Y("word:N").title(None),
    alt.Tooltip(['season', 'word', 'count', 'normalized_count']),
    alt.Color("normalized_count", scale=alt.Scale(scheme='yelloworangebrown', domain=[0.001,2])).title("Count per 1,000 words"),
).properties(width=500, height=700, title=f'Survivor: Counts of selected keywords, by season').configure_legend(orient='top', gradientLength=170).configure_axis(
    labelFontSize=10,
)

alt.Chart(...)

---

## Exports

#### Save the aggregated results to JSON

In [15]:
season_agg_df.to_json('../data/processed/transcripts/season_keyword_counts.json', orient='records', indent=4)
series_agg_df.to_json('../data/processed/transcripts/series_keyword_counts.json', orient='records', indent=4)

#### Save the results in a nested JSON file

In [16]:
output_path = '../data/processed/transcripts/transcript_keyword_counts.json'
with open(output_path, 'w') as f:
    json.dump(results, f, indent=4)